# Multi Language Integration

**Author:** Nico Curti

**Course:** Software and Computing for Applied Physics - 87948

**Github:** [Nico-Curti](https://github.com/Nico-Curti)

All the programming languages can be broadly split into two main classes:

* Low level programming languages
* High level programming languages

The "height" of the language is evaluated in relation to the "distance" between programmer and CPU.

As conseguence, low level languages are typically harder to write **but** they provide higher degree of freedoms and higher performances if used correctly.

In contrary, high level languages are typically simpler to write **but** they provide lower degree of freedoms and lower performances, even if used correctly.

Or, in other words...

    "With great power comes great responsibility"
    (cit. Uncle Ben)

As example

* Low level programming languages $\rightarrow$ Python

* High level programming languages $\rightarrow$ C++

### An Example

In [9]:
import numpy as np
from scipy.spatial.distance import euclidean

def point_distance (N):
    pt = np.random.uniform(0, 1, size=(N, 2))    
    for i in range(N):
        for j in range(N):
            x1, y1 = pt[i]
            x2, y2 = pt[j]
            d = euclidean([x1, y1], [x2, y2])

%timeit point_distance(N=1000)

6.63 s ± 18.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


```cpp
#define N 1000
int main () {
    std :: unique_ptr < std :: tuple < float, float> [] > pt(new std :: tuple < float, float >[N]);    
    std :: mt19937 gen(42);
    std :: uniform_real_distribution < float > unif_d(0.f, 1.f);
    std :: generate_n(pt.get(), N, [&]{ return std :: pair(unif_d(gen), unif_d(gen)); });
    
    for (int i = 0; i < N; ++i)
        for (int j = 0; j < N; ++j) {
            float x1 = std :: get <0>(pt[i]); 
            float y1 = std :: get <1>(pt[i]);
            float x2 = std :: get <0>(pt[j]); 
            float y2 = std :: get <1>(pt[j]);
            float d = std :: sqrtf((x1 - x2) * (x1 - x2) + (y1 - y2) * (y1 - y2));
        }
    return 0;
}
```

```python
>>> 0.04 ms
```

Ok but Python is so nice...

## Some notes about optimization

    “Premature optimization is the root of all evil” (cit. Donald Knuth)

1. Re-think your implementation, trying to optimize it
2. Use built-in libraries
3. Check if all the operations are usefull or not
4. Optimize the memory accesses: less memory typically leads to poor performances, more memory is often the best approach so do not be greedy!

## Re-think your implementation

We have already seen how the Numpy approach could be very useful to speedup your code and also improve the readability of your snippets.

* [x] Generate the points using numpy functions

```python
pt = np.random.uniform(low=0, high=1, size=(N, 2))
```

* [ ] We need to evaluate distance and we have already used the scipy function for pairs of points.
    Could be useful to check the documentation of this function...

In [2]:
help(euclidean)

Help on function euclidean in module scipy.spatial.distance:

euclidean(u, v, w=None)
    Computes the Euclidean distance between two 1-D arrays.
    
    The Euclidean distance between 1-D arrays `u` and `v`, is defined as
    
    .. math::
    
       {\|u-v\|}_2
    
       \left(\sum{(w_i |(u_i - v_i)|^2)}\right)^{1/2}
    
    Parameters
    ----------
    u : (N,) array_like
        Input array.
    v : (N,) array_like
        Input array.
    w : (N,) array_like, optional
        The weights for each value in `u` and `v`. Default is None,
        which gives each value a weight of 1.0
    
    Returns
    -------
    euclidean : double
        The Euclidean distance between vectors `u` and `v`.
    
    Examples
    --------
    >>> from scipy.spatial import distance
    >>> distance.euclidean([1, 0, 0], [0, 1, 0])
    1.4142135623730951
    >>> distance.euclidean([1, 1, 0], [0, 1, 0])
    1.0



Can we use it to compute the distances between the entire list of points?

If the documentation is not clear enough, we can always check it ourself!

**This is one of the most useful feature of high-level programming: an easy and fast way to check our idea**

In [10]:
N = 10
pt = np.random.uniform(low=0, high=1, size=(N, 2))

euclidean(pt, pt)

ValueError: Input vector should be 1-D.

Ok, it doesn't work... but maybe there are other function that can address the same purpose

Check [scipy.spatial](https://docs.scipy.org/doc/scipy/reference/spatial.html) documentation

In [5]:
from scipy.spatial import distance_matrix

help(distance_matrix)

Help on function distance_matrix in module scipy.spatial._kdtree:

distance_matrix(x, y, p=2, threshold=1000000)
    Compute the distance matrix.
    
    Returns the matrix of all pair-wise distances.
    
    Parameters
    ----------
    x : (M, K) array_like
        Matrix of M vectors in K dimensions.
    y : (N, K) array_like
        Matrix of N vectors in K dimensions.
    p : float, 1 <= p <= infinity
        Which Minkowski p-norm to use.
    threshold : positive int
        If ``M * N * K`` > `threshold`, algorithm uses a Python loop instead
        of large temporary arrays.
    
    Returns
    -------
    result : (M, N) ndarray
        Matrix containing the distance from every vector in `x` to every vector
        in `y`.
    
    Examples
    --------
    >>> from scipy.spatial import distance_matrix
    >>> distance_matrix([[0,0],[0,1]], [[1,0],[1,1]])
    array([[ 1.        ,  1.41421356],
           [ 1.41421356,  1.        ]])



In [13]:
N = 5
pt = np.random.uniform(low=0, high=1, size=(N, 2))

distance_matrix(pt, pt)

array([[0.        , 0.38862428, 0.46907901, 0.40125491, 0.76886231],
       [0.38862428, 0.        , 0.58908841, 0.69620029, 0.94636221],
       [0.46907901, 0.58908841, 0.        , 0.28631425, 0.35745467],
       [0.40125491, 0.69620029, 0.28631425, 0.        , 0.40968752],
       [0.76886231, 0.94636221, 0.35745467, 0.40968752, 0.        ]])

Check if it works better now

In [14]:
import numpy as np
from scipy.spatial import distance_matrix

def point_distance (N):
    pt = np.random.uniform(0, 1, size=(N, 2))    
    d = distance_matrix(pt, pt)

%timeit point_distance(N=1000)

40.3 ms ± 389 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


With a simple check of the online documentation we moved from 6.63s to 40.3ms

Great job!

But the low level implementation is still better...

We have spent more memory in the allocation of the entire matrix...

Can we do better than this?!

## Some notes about optimization

    “Premature optimization is the root of all evil” (cit. Donald Knuth)

1. [x] Re-think your implementation, trying to optimize it
2. [x] Use built-in libraries
3. [ ] **Check if all the operations are usefull or not**
4. [x] Optimize the memory accesses: less memory typically leads to poor performances, more memory is often the best approach so do not be greedy!

In [15]:
N = 5
pt = np.random.uniform(low=0, high=1, size=(N, 2))

distance_matrix(pt, pt)

array([[0.        , 0.36857021, 0.23499753, 0.34164238, 0.53563521],
       [0.36857021, 0.        , 0.60320484, 0.3895317 , 0.16707837],
       [0.23499753, 0.60320484, 0.        , 0.49651218, 0.77016262],
       [0.34164238, 0.3895317 , 0.49651218, 0.        , 0.50861513],
       [0.53563521, 0.16707837, 0.77016262, 0.50861513, 0.        ]])

### DAMN IT! The matrix is symmetric

There are at least $\frac{N\times(N-1)}{2}$ useless operations..

A better approach could involve the evaluation of only the interesting parts, without any redundancy

In [18]:
from scipy.spatial.distance import pdist

help(pdist)

Help on function pdist in module scipy.spatial.distance:

pdist(X, metric='euclidean', *, out=None, **kwargs)
    Pairwise distances between observations in n-dimensional space.
    
    See Notes for common calling conventions.
    
    Parameters
    ----------
    X : array_like
        An m by n array of m original observations in an
        n-dimensional space.
    metric : str or function, optional
        The distance metric to use. The distance function can
        be 'braycurtis', 'canberra', 'chebyshev', 'cityblock',
        'correlation', 'cosine', 'dice', 'euclidean', 'hamming',
        'jaccard', 'jensenshannon', 'kulsinski', 'kulczynski1',
        'mahalanobis', 'matching', 'minkowski', 'rogerstanimoto',
        'russellrao', 'seuclidean', 'sokalmichener', 'sokalsneath',
        'sqeuclidean', 'yule'.
    **kwargs : dict, optional
        Extra arguments to `metric`: refer to each metric documentation for a
        list of all possible arguments.
    
        Some possibl

In [19]:
N = 5
pt = np.random.uniform(low=0, high=1, size=(N, 2))

pdist(pt)

array([0.66425941, 0.37999825, 0.54942056, 1.05839018, 0.79106797,
       0.40562971, 0.39510475, 0.85226327, 1.14105877, 0.72397826])

Check if it works better now

In [20]:
import numpy as np
from scipy.spatial.distance import pdist

def point_distance (N):
    pt = np.random.uniform(0, 1, size=(N, 2))    
    d = pdist(pt)

%timeit point_distance(N=1000)

2.82 ms ± 36.9 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Again, with a simple check of the online documentation we moved from 6.63s to 2.82ms

Great(er) job!

But, again, the low level implementation is still better...

Can we do better than this?!

## Why Numpy is so fast?!

All the high level programming languages relies on a backend written in low level programming languages.

In this way they can achieve valuable computational performances, simplifing the user interface and providing ready-to-use solutions

As example

* Low level programming languages $\rightarrow$ Python $\rightarrow$ prototyping

* High level programming languages $\rightarrow$ C++ $\rightarrow$ efficiency

## How can we combine low and high levels?

Real world applications and codes involves the integration and usage of several programming languages together.

The idea is that, despite we can potentially use a unique language to perform all our taks, there is always a better/faster language to perform a particular aim.

As example:

* Python language is internally written in C++

* When you want to write a code you probably use an IDE which is developed in other languages (like Python)

* When you want to execute a script you start from the command line, which supports languages like Bash or Powershell

* If you want to use a compiler for C++, you are using an executable built from a C++ script

* This presentation is showed as HTML file, which is a markup language, in which the animations are rendered via Javascript (another language) and the style/font/formats delegated to CSS (another language)

## Integrate C++ into Python

Fortunately, also in this case there are several ready-to-use packages and techniques to achieve this purpose.

The most famous are certainly:

* [pybind11](https://github.com/pybind/pybind11)
* [cython](https://github.com/cython/cython)

## Cython

Cython is a Python compiler that makes writing C extensions for Python as easy as Python itself. Cython is based on Pyrex, but supports more cutting edge functionality and optimizations.

Cython translates Python code to C/C++ code, but additionally supports calling C functions and declaring C types on variables and class attributes. This allows the compiler to generate very efficient C code from Cython code.

This makes Cython the ideal language for wrapping external C libraries, and for fast C modules that speed up the execution of Python code.

```bash
python -m pip install Cython
```

**NOTE:** The main difference between Python and C++ is the presence of a *compiler*

Python is an interpretated language which does not require any preliminary step or check before its usage!

C++ is a compiled language which must be compiled before its usage!

## Example

Our purpose is to write a very simple function in Python which takes a string of a "Name" and it returns the classical output

```bash
Hello "Name"
```

We want a very efficient way to address this task, so the string building will be performed using C++ function.

### Requirements

* A C++ function which takes a string as input and returns a string as output

```c++
std :: string HelloWorld (const std :: string & name) {
    std :: string msg = "Hello " + name;
    return msg;
}
```

* A Python function which takes a string as input, it processes the string, and it returns a string as output.

```python
def HelloWorld (name : str) -> str:
    output = CppFunc(name)
    return output
```

* A way to compile the C++ function, making it visible from the Python counter part

Here is where the magic happens and the use of **Cython** makes all these steps possible and easy to use!

For the integration of cython in simple projects you can directly use the *cythonize* function provided by the package

**helloworld.pyx**
```python
print("Hello World")
```

**setup.py**
```python
from setuptools import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize('helloworld.pyx')
)
```

**cmd compiler**
```bash
$ python setup.py build_ext --inplace
```

**Python**
```python
>>> import helloworld
Hello World
```

In our case we want to perform something more advanced, involving the passing of variables between Python and C++ and back.

In this case is very useful the integration of the [scikit-build](https://scikit-build.readthedocs.io/en/latest/) package, which provides a "standardized" way to integrate the two languages...

... involving the insertion of a third language, i.e. [CMake](https://cmake.org/)

## A step-by-step guide for your first multi-languages integration

```bash
sktest/
├── CMakeLists.txt
├── LICENSE
├── MANIFEST.in
├── README.md
├── SOURCES.txt
├── cython_hello_world
│   ├── __init__.py
│   ├── __version__.py.in
│   ├── lib
│   │   ├── __init__.pxd
│   │   └── hello.pxd
│   └── source
│       ├── CMakeLists.txt
│       └── hello.pyx
├── hpp
├── include
│   └── hello.h
├── pyproject.toml
├── requirements.txt
├── setup.py
└── src
    └── hello.cxx

6 directories, 16 files
```

### C++ components

**`include/hello.h`**
```c++
#ifndef __hello_h__
#define __hello_h__

#include <iostream>

namespace test
{
	/**
	* @brief Dummy function
	*
	* @details Dummy Hello World function in C++
	* to combine two strings.
	*
	* @param name Input name to hello.
	* @return msg Hello message.
	*
	*/
	std :: string HelloWorld (const std :: string & name); 
	
} // end namespace test

#endif // __hello_h__
```

**`src/hello.cxx`**
```c++
#include <hello.h>

namespace test
{
	std :: string HelloWorld (const std :: string & name)
    {
        std :: string msg = "Hello " + name;
        return msg;
    }

} // end namespace test
```

Some **NOTE**:

1. The declaration of the functions must be put in an header file with the .h extension
2. The implementation of those functions must be put in a source file with the .cxx, .c, .cpp extension
3. If there are template function which must be compiled according to their usage in the source file, you can put them into a special header file with the extension .hpp

Moreover:
    
1. The current documentation of the functions follows the Doxygen format, which is a standard for C++ code and it can be easily integrated also in Python (see later explanations)
2. We have introduced also a namespace for a complete generalization of the current example
3. The use of \_\_magic\_\_ define is a best practice to avoid multiple inclusions of the same definitions and errors in larger project

### Cython components

**`cython_hello_world/lib/hello.pxd`**
```python
# distutils: language = c++
# cython: language_level=2

# import the wrapped Cython type from the
# cython std library
from libcpp.string cimport string

# define the import of the CXX functions from the
# header files, following the syntax:
#
# cdef extern from "header.h" [namespace "std"] [nogil]:

cdef extern from "hello.h" namespace "test" nogil:

	string HelloWorld (const string & name) except +;
```

**`cython_hello_world/lib/__init__.pxd`**
```python
# distutils: language = c++
# cython: language_level=2

cimport hello
```

**`cython_hello_world/source/hello.pyx`**
```python
# distutils: language = c++
# cython: language_level=2

# import the declaration of the CXX function from the pxd header file
from hello cimport HelloWorld

__author__ = ['Nico Curti']
__email__ = ['nico.curti2@unibo.it']

# NOTE: the typing of the input variables
# and returns boosts the efficiency of the
# Cython wrap! Use it as much as possible...
def _HelloWorld (str name) -> str:
	# NOTE: the management of the string between
	# Cython and CXX is not straightforward!
	# For the correct forward/backward transmission
	# you need to convert them in Bytes using the
	# correct encoding... (std way is utf-8 fmt)
	cdef bytes _name = name.encode('utf-8')
	cdef bytes res = HelloWorld(_name)

	return res.decode('utf-8')
```

Some **NOTE**:

1. The declaration of the functions must be put in an header file with the .pxd extension
2. The implementation of those functions must be put in a source file with the .pyx extension
3. The \_\_init\_\_.pxd file has the same properties of the standard \_\_init\_\_.py file and it provides the visibility to the functions that can be exported.
    In this case it is used to guarantee the visibility of the other header files during the compilation and **cimport**ing

### Python components

**`cython_hello_world/__version__.py.in`**
```python
#!/usr/bin/env python
# -*- coding: utf-8 -*-

__author__  = ['Nico Curti']
__email__ = ['nico.curti2@unibo.it']

__version__ = '@PACKAGE_VERSION@'
```

**`cython_hello_world/__init__.py` (part 1)**
```python
import os

# absolute path of the current file
# NOTE: using the magic symbol __file__ you can
# ensure the correctness of this path in any location in
# which the script is called!
path = os.path.dirname(__file__)

try:
    # try to import the Cython submodules of the package
    from .hello import _HelloWorld as HelloWorld

except ImportError as e:
    import ctypes
    import warnings
    from glob import glob
```

**`cython_hello_world/__init__.py` (part 2)**
```python
    # all these issues can be solved setting the LD_LIBRARY_PATH env
    # variable to the correct path
    warnings.warn( (
        'Load of the dynamic library failed. '
        f'To avoid this issue you must add to the LD_LIBRARY_PATH the path:{path} '
        'Trying to solve this issue with a dynamic loading via ctypes '
    ),
    category=UserWarning
    )

    # set the library name
    # NOTE: you can change this line and re-use the entire
    # code (with the appropriated import...) in any other project
    LIBRARY_NAME = 'cython_utils'
    # search all the occurrences with that name in a library format
    LIBRARY_PATH = glob(f'{path}/*{LIBRARY_NAME}[.-]*')
```

**`cython_hello_world/__init__.py` (part 3)**
```python
    if len(LIBRARY_PATH) != 1:
        # there are multiple locations with the required name
        # or there are no location found
        raise ImportError('Failed to load the dynamic library')

    # get the first item of the list
    LIBRARY_PATH = LIBRARY_PATH[0]
    # load the dynamic library
    ctypes.cdll.LoadLibrary(LIBRARY_PATH)

    # now you can correctly perform the import
    from .hello import _HelloWorld as HelloWorld
```

Some **NOTE**:

1. The \_\_version\_\_ file is used as "initial" file for the automated creation of the correct version (note the *.in* extension).
    In particular, the parameters set with the name between @ symbols will be filled during the source file compilation.
2. The \_\_init\_\_ file represents the core of this script and it is used to set the visibility of the cythonized functions and their export to standard python code.
    In particular, the cython proxy function is replaced with an alias without underscore: this is used to avoid misunderstanding between the cython and python version of the same functions!
3. All the notes about the libraries and other import managements are set to face the OS compatibility issues that can occur in terms of output file positioning

## Now we have all the elements ready and we need to compile the C++ components

## CMake

CMake is the de-facto standard for building C++ code (and other languages!)

1. Advanced workflow management
2. Make is for Unix-like architecture, while CMake works in any OS environment
3. It has its own syntax, allowing an easy way to manage also the filesystem

### Minimal example (C++)

```cmake
cmake_minimum_required(VERSION 3.1...3.29)
project(ProjectName VERSION 1.0 LANGUAGES CXX)
add_executable(ExecExample simple_example.cpp)
```

In our case we need to build a more advanced CMake file (which is commonly identified with the name *CMakeLists.txt*) since we are interested in building scripts with the support of Cython!

**`CMakeLists.txt` (part 1)**
```cmake
cmake_minimum_required(VERSION 3.18.22)

# NOTE: the variable PACKAGE_VERSION and CMAKE_PROJECT_DESCRIPTION
# are fed by the command line built by the setup.py script
project(cython_hello
  LANGUAGES CXX
  VERSION ${PACKAGE_VERSION}
  DESCRIPTION ${CMAKE_PROJECT_DESCRIPTION}
)
enable_language(C) # this is mandatory for scikit-build check
enable_language(CXX)
```

**`CMakeLists.txt` (part 2)**
```cmake
# mandatory script of the build
find_package(Cython REQUIRED)
find_package(PythonExtensions REQUIRED)

# get the full list of scripts to build
file(GLOB SOURCE_CXX  "${CMAKE_CURRENT_LIST_DIR}/src/*.cxx"  )
file(GLOB HEADER_CXX  "${CMAKE_CURRENT_LIST_DIR}/include/*.h")

# add the include directory to the list of paths in which
# look for the -I cmd
include_directories("${CMAKE_SOURCE_DIR}/include")
```

**`CMakeLists.txt` (part 3)**
```cmake
# set the name of the utils library to build
# NOTE: the utils library in this case stores
# all the CXX scripts wrapped by Cython
set (cython_utilslib cython_utils)

# add the library to the list of builds, including
# all the .cxx, .h, .hpp files as dependencies
add_library(${cython_utilslib} SHARED ${SOURCE_CXX} ${HEADER_CXX})

# install the cython_utils library in the same location
# of the final Python package
# (e.g. /.local/lib/python3.10/site-packages/cython_hello_world)
install(TARGETS ${cython_utilslib}
  LIBRARY DESTINATION cython_hello_world
  RUNTIME DESTINATION cython_hello_world
)

# move to the cython CMakeFile sub-directory
add_subdirectory("${CMAKE_CURRENT_SOURCE_DIR}/cython_hello_world/source")
```

In this way we have set all the variable paths dedicated for the description of the C++ source codes.

**NOTE:** CMake provides a long list of ready-to-use variables for the management of standard paths.
    
All these variables are identified by the prefix `CMAKE_*`
    
A complete list of them can be found [here](https://cmake.org/cmake/help/latest/manual/cmake-variables.7.html#variables-that-provide-information)

In the above file we have "delegated" the management of the Cython scripts to a second CMake file, stored in the `cython_hello_world/source` directory.

Let's see how we can define this second file...

**`cython_hello_world/source/CMakeLists.txt` (part 1)**
```cmake
# automatically fill the __version__.py script setting
# the variable defined as @VAR@
# NOTE: this is the standard way in which we can customize
# a script according to runtime build.
configure_file(
  "${CMAKE_SOURCE_DIR}/cython_hello_world/__version__.py.in"
  "${CMAKE_SOURCE_DIR}/cython_hello_world/__version__.py"
  @ONLY
)

# Include the *.pxd directory
include_directories("${CMAKE_SOURCE_DIR}/cython_hello_world/lib")

# add a new target for the Cython builder
# setting the language of the build as CXX
add_cython_target(hello CXX)
```

**`cython_hello_world/source/CMakeLists.txt` (part 2)**
```cmake
# add the Cython library, configuring it as a module
# of the resulting Python package
# NOTE: this line creates an association between the target
# and a new module with a custom name
add_library(hello MODULE "${hello}")
# setting the module for the final Python package
python_extension_module(hello)

# install the compiled Cython target in the same location
# of the final Python package
# NOTE: the resulting module will be built as a CXX library,
# so a binary file in fmt .so, .dll according to the OS
install(TARGETS hello LIBRARY DESTINATION cython_hello_world)
# dynamic link the Cython library to the pre-built CXX utility
# library
# NOTE: This line is very important for the correct usage of
# the final package and it requires a carefully management of
# the imports and environment variables!!
# (see cython_hello_world/__init__.py file for a detailed description
# of the issues and their possible workarounds...)
target_link_libraries(hello ${cython_utilslib})
```

**NOTE:** The possibility to fill configuration file during compile time allows to easily generalize the scripts keeping track of the versioning of our package!

The output of our compiler will be a library (aka `.so` file) which will contain all the instructions compatible with the Python interpreter.

**Great language!**

## Steps

* [x] write the C++ code
* [x] write the Cython code
* [x] write the instruction to compile our cython code with C++ compiler
* [x] integrate the compiler with CMake support

* [ ] build the setup file for the correct building of the **Python** counter part!

**`setup.py` (part 1)**
```python
#!/usr/bin/env python
# -*- coding: utf-8 -*-

import warnings
from skbuild import setup

PACKAGE_NAME = 'cython_hello_world'
AUTHOR = 'Nico Curti'
EMAIL = 'nico.curti2@unibo.it'
REQUIRES_PYTHON = '>=3.5'
KEYWORDS = 'scikit-build cython examples dummy'
PACKAGE_VERSION = '0.0.1'
DESCRIPTION = 'MWE for Cython build with scikit-build support'
```

**`setup.py` (part 2)**
```python
setup(
    name=PACKAGE_NAME,
    version=PACKAGE_VERSION,
    description=DESCRIPTION,
    author=AUTHOR,
    author_email=EMAIL,
    maintainer=AUTHOR,
    maintainer_email=EMAIL,
    python_requires=REQUIRES_PYTHON,
    keywords=KEYWORDS,
    packages=[PACKAGE_NAME,],
    package_data={PACKAGE_NAME: ['__version__.py.in',],},
    cmake_install_dir='',
    cmake_args=[
      '-DCMAKE_BUILD_TYPE:STRING=Release',
      f'-DPACKAGE_VERSION:STRING={PACKAGE_VERSION}',
      f'-DCMAKE_PROJECT_DESCRIPTION:STRING={DESCRIPTION}',
    ],)
warnings.warn( (
  'Dynamic libraries added to the installation path. '
  'Please add to the LD_LIBRARY_PATH the current installation path to avoid '
  'possible ImportError exception '
  ),
  category=UserWarning)
```

Some **NOTE**:

1. The definition of the variables in the header of the setup script allows a fast editing of the information
2. Further details about our package can be inserted into a dedicated `pyproject.toml` file, which follows the `ini` format
3. The most important thing is the insertion of the generated library into the `LD_LIBRARY_PATH` environment variable.
    To prevent possible user issue, a dedicated warning is raised at the end of the installation!
    This could be avoided setting the installation of the Cython library in the standard library path, but it is a practice useful **only** for the release phase!

Now everything is ready and we can simply build all our project with the command

```bash
$ python -m pip install . -v
```

# Summary

1. `C++` language for the source code definition and the "real" computation
2. `CMake` language for the building of the C++ codes, providing a solution independent from the OS
3. `Cython` language for the wrapping of the C++ code
4. `Scikit-build` package for the integration of the CMake into Python
5. `Python` for the setup of the package and final usage of the code
6. `Bash`/`Powershell` for the run of the scripts

**Extra:** we used Doxygen format for the declaration of the C++ information

The reward of all this work:

```python
>>> from cython_hello_world import HelloWorld
>>> HelloWorld('Nico')
'Hello Nico'
```

**GREAT JOB!!**

## And the vice versa??

### Is it possible to integrate Python into C++?

The Python language **is** a C++ code... so the answer is certainly **YES**

There are also several libraries very interesting and useful which aim to address this task

```c++
#define PY_SSIZE_T_CLEAN
#include <Python.h>

int
main(int argc, char *argv[])
{
    wchar_t *program = Py_DecodeLocale(argv[0], NULL);
    if (program == NULL) {
        fprintf(stderr, "Fatal error: cannot decode argv[0]\n");
        exit(1);
    }
    Py_SetProgramName(program);  /* optional but recommended */
    Py_Initialize();
    PyRun_SimpleString("from time import time,ctime\n"
                       "print('Today is', ctime(time()))\n");
    if (Py_FinalizeEx() < 0) {
        exit(120);
    }
    PyMem_RawFree(program);
    return 0;
}
```

Other interesting libraries:

* [matplotlib-cpp](https://github.com/lava/matplotlib-cpp) Possibly the simplest C++ plotting library.
* [NumCpp](https://github.com/dpilger26/NumCpp) A Templatized Header Only C++ Implementation of the Python NumPy Library